# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk import word_tokenize
from nltk import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import pickle 
import re 
import nltk
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

In [3]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
print(stopwords.words('english'))
 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///disaster_project.db')

sql = 'select * from cleaned_messages'
#table is cleaned_messages
df = pd.read_sql_query(sql,engine)
y = df.drop('message',axis=1) 
y = y.drop('id',axis=1) 
y = y.drop('original',axis=1) 
y = y.drop('genre',axis=1)
y = y.drop('child_alone',axis=1) #no values are populated in the data
y = y.astype('int')
X = df['message'] 

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#add to X, make dummies. could this improve the model? 
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    '''removes non alphanumeric characters from text input parameter, then returns that cleaned text'''
    text = re.sub(r'[^a-zA-Z0-9\s]',' ',text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = []
    for tok in tokens:
        clean_tok = WordNetLemmatizer().lemmatize(tok,).lower().strip()
        cleaned_tokens.append(clean_tok)
        
    return cleaned_tokens
        
    

In [7]:
toktest = tokenize(df['message'].iloc[0])
toktest

['weather',
 'update',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

In [8]:
nas = df.isna().sum()
nas[nas>0]

original    16046
dtype: int64

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
#multi model pipelines with different ML models.  
pipe_lr = Pipeline([('vect',CountVectorizer(tokenizer=tokenize,token_pattern=None)),
                    ('Tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier( LogisticRegression(max_iter=1000,random_state=42)))])


pipelist = [pipe_lr]


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [11]:
def fit_your_pipeline(modelname,x_train,y_train):
    '''a definition to insert in a loop to fit multiple models 
    inputs: 
    the name of the model
    the X_train data
    the y_train_data
    output the fitted model. 
    there is no need for a return statement, but it prints out when done. 
    NOTE: this can take some time.'''
    try:
        modelname.fit(x_train, y_train)
        print(f'fitted model: {modelname}') 
    except Exception as e:
        print(f'exception for model: {modelname} of {e}')  

In [12]:
#this takes some time to run. Grab a cup of your favorite beverage. 
#pipeline.fit(X_train,y_train)
for i in range(len(pipelist)):
    fit_your_pipeline(pipelist[i],X_train,y_train)

fitted model: Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x000001F3E7E404C0>)),
                ('Tfidf', TfidfTransformer()),
                ('rf',
                 MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000,
                                                                    random_state=42)))])


In [13]:
pipelist = [pipe_lr]

In [14]:
import joblib
for i in range(len(pipelist)):
    pipe = pipelist[i]
    temp2 = pipe.steps[-1][1].estimator.__class__.__name__
    print(temp2)
    #model_name = type(temp[2]).__name__
    #print(model_name)
    #name = str(pipe)
    #print(pipelist[i])
    joblib.dump(pipe, f'{str(temp2)}.pkl')
#joblib.dump(pipe, 'pipe_lr.pkl')

    

LogisticRegression


In [16]:
y_pred = pipe_lr.predict(X_test)


In [17]:
y_pred.shape

(5244, 35)

In [18]:
ycols = y_test.columns.tolist()
y_pred2 = pd.DataFrame(y_pred,columns=ycols)
#y_test[ycols[i]]
type(y_pred2)

pandas.core.frame.DataFrame

In [19]:
y_pred2.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
print(y_test.columns)
print(y_pred2.columns)
print(set(y_test.columns)-set(y_pred2.columns))

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')
Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'f

In [21]:
#TODO need to add the F1 score too. 
#TODO convert this to a definition 
#note: need a division by zero catch. 
l = []
for i in range(len(ycols)):
    
    accuracy=accuracy_score(y_test[ycols[i]],y_pred2[ycols[i]])
    precision=precision_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted',zero_division=1)
    recall=recall_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted')
    f1score = f1_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted')    
    temp ={'Column':[ycols[i]]
                     ,'Accuracy':[accuracy]
                     ,'Precision':[precision],
                     'Recall':[recall],
                      'F1 Score':[f1score]}
    
    temp = pd.DataFrame(temp)
    #temp = {k:[v] for k,v in temp.items()}  
    l.append(temp)
print(len(l))
dfmetrics=pd.concat(l,axis=0)
dfmetrics.to_excel('metrics_file.xlsx')
dfmetrics


35


,Column,Accuracy,Precision,Recall,F1 Score
0,related,0.820938,0.809149,0.820938,0.805810
0,request,0.898169,0.892883,0.898169,0.890800
0,offer,0.994851,0.994878,0.994851,0.992284
0,aid_related,0.779748,0.779027,0.779748,0.776977
0,medical_help,0.921434,0.899093,0.921434,0.897901
0,medical_products,0.956903,0.949780,0.956903,0.942779
0,search_and_rescue,0.975400,0.972574,0.975400,0.965500
0,security,0.983028,0.983316,0.983028,0.974615
0,military,0.968726,0.959574,0.968726,0.956609
0,water,0.958810,0.954496,0.958810,0.953733


In [24]:
#looking at average accuracy, precision, recalls since over 30 output variables. 
summary = dfmetrics.describe()

In [25]:
summary.to_excel('metrics_summary_file.xlsx')